#Training
Todos nuestros archivos propios (TFrecord, pbtxt) se subieron a google drive, por lo que configuramos colab para tener acceso al drive.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Se instaló la API de Tensorflow que incluía a *model zoo*, entre otras herramientas. Tambíen se instalaron otros paquetes por medio de pip como *dependencies*.

In [0]:
!git clone https://github.com/tensorflow/models.git
import sys
sys.path.append('models/research')
sys.path.append('models/research/object_detection')

In [0]:
!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

A continuación se descarga el modelo preentrenado para el modelo que elegimos, faster_rcnn_inception_v2_coco

In [0]:
%cd /content
import os
import shutil
import glob
import urllib
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

Editamos el archivo .config con nuestros parámetros. Se cambia el trainining record, el validation record, el label map, el modelo inicial (este viene siendo el descargado en el paso anterior) y el número de clases.

In [0]:
import re

filename = '/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/pretrained_model/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/content/gdrive/My Drive/train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/content/gdrive/My Drive/test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/content/gdrive/My Drive/label_map.pbtxt', s)
  s = re.sub('num_classes: 90', 'num_classes: 13', s)
  f.write(s)

Se corre el script que entrena la red.

In [0]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path=models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --model_dir=trained \
    --alsologtostderr \
    --num_train_steps=3000 \
    --num_eval_steps=500

Se guarda el último checkpoint guardado para a partir de ahí general el frozen model que se le dará a nuestra aplicación.

In [0]:
import os
lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --output_directory='/content/gdrive/My Drive/fine_tuned_model' \
    --trained_checkpoint_prefix=trained/$last_model

Las siguientes herramientas sirven para poder usar Tensorboard.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

A esto le damos el directorio donde se entrenó el modelo y al correr la celda nos arrojará una liga donde podemos ver el Tensorboard del entrenamiento.

In [0]:
LOG_DIR = 'trained'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"